In [4]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import glob

def decay(x,a,b,c):
    #c = tau    
    return a + b*np.exp(-x/c)

data_path = 'ringdown raw data/11-05-2021 633/20211105-0002/'
data_files = glob.glob(data_path + '*.csv')
ring_downs = []
raw_ring_downs = []

for i in data_files:
    data = pd.read_csv(i, skiprows=1) 
    start_full = data.loc[(data['(V)'] == max(data['(V)']))].last_valid_index()
    end_full = data.loc[(data['(us)'] >= 350)].first_valid_index()
    
    data2 = data[start_full:end_full]
    
    #need to do 0.9 and 0.1 times the voltage and find the times for those
    
    #determine minimum V to get full range
    vrange = max(data2['(V)']) - min(data2['(V)'])
    vmax = max(data2['(V)']) - (0.1 * vrange)
    vmin = min(data2['(V)']) + (0.1 * vrange)
    
    start = data2.loc[(data2['(V)'] <= vmax)].first_valid_index()
    end = data2.loc[(data2['(V)'] <= vmin)].first_valid_index()

    time = data['(us)']
    time = time[start_full:end_full]
    voltage = data['(V)']
    voltage = voltage[start_full:end_full]
    
    popt, pcov = curve_fit(decay, time, voltage)
    
    residuals = voltage - decay(time, *popt)
    res_ss = np.sum(residuals**2)
    ss_tot = np.sum((voltage - np.mean(voltage))**2)
    r_squared = 1 - (res_ss / ss_tot)
    
    raw_time = data2['(us)']
    raw_time = raw_time[start:end]
    raw_ringdown = max(raw_time)-min(raw_time)
    
    limit = 0.998
    if r_squared >= limit:
        ring_downs.append(popt[2]*2.198)
        raw_ring_downs.append(raw_ringdown)
    else:
        print('Rejecting poor fit.')
        

print('Minimum R-Squared: ', limit)    
print('Successful ringdown fits: ', len(ring_downs),)
print('Average calculated ringdown time: ', np.mean(ring_downs), ' (us)')
print('Average raw ringdown time: ', np.mean(raw_ring_downs), ' (us)')
print('Calculated ringdown Standard Deviation: ', np.std(ring_downs), ' (us)')
print('Raw ringdown Standard Deviation: ', np.std(raw_ring_downs), ' (us)')


#Can't reproduce the data generated by the picoscope software exactly, 3us off roughly for some reason and standard deviation also slightly off.



Minimum R-Squared:  0.998
Successful ringdown fits:  32
Average calculated ringdown time:  53.95886711095321  (us)
Average raw ringdown time:  55.54824842812499  (us)
Calculated ringdown Standard Deviation:  2.920284065424433  (us)
Raw ringdown Standard Deviation:  3.4043479595839568  (us)
